# Crypto / √163

The main reference here I used is [CSI-FiSh](https://eprint.iacr.org/2019/498.pdf), but I don't fully understand everything, so take it with a grain of salt. First set up our $p$.

In [1]:
ells = [*primes(3, 128), 163]
p = 4 * prod(ells) - 1

PARI has a `quadclassunit` for computing the class group of a quadratic order of discriminant D.

In [2]:
%time pari.quadclassunit(-4*p)

CPU times: user 13min 49s, sys: 1.86 s, total: 13min 51s
Wall time: 13min 46s


[102019419125180345266808265, [102019419125180345266808265], [Qfb(18673762890019598786089492, -7970720562485151774511690, 70933886576327064156523437)], 1]

Basically, this result says that the class group is cyclic of order 102019419125180345266808265. Let's first check this using the ideal corresponding to the 3-isogeny. We also use binary quadratic forms because they're equivalent somehow (see Cohen's course in algebraic ANT).

In [3]:
order = 102019419125180345266808265

q3 = pari.Qfb(3, 2, (p + 1) // 3)
assert q3^order == q3^0

In fact, this turns out to be a generator, so even better! Let's check this by exponentiating it against the other possible orders.

In [4]:
next(i for i in divisors(order) if q3^i == q3^0)

102019419125180345266808265

Also, by a stroke of luck this value is smooth:
```factor(order) = 3^2 * 5 * 13^2 * 14153 * 130241 * 7277586888541```

So our plan of attack is not to understand binary quadratic forms (or really any theory) at all, but just take discrete logs of everything instead. It's kinda suboptimal but whatever.

In [5]:
from tqdm import tqdm

dlogs = []
for ell in tqdm(ells[1:]):
    qfb = pari.Qfb(ell, 2, (p + 1) // ell)
    dlog = discrete_log(qfb, q3, order, operation=None, identity=q3^0, inverse=lambda x:x^-1, op=lambda a,b:a*b)
    dlogs.append(dlog)
print(f'{dlogs = }')

100%|███████████████████████████████████████████████████████████████████████████████████| 30/30 [17:44<00:00, 35.49s/it]

dlogs = [99277373102719610806018318, 83421842872387889334089797, 56025662040177681113282671, 49118499040862518947572689, 15645397576648648377412663, 61665764328525124260421806, 40759403716184792525659432, 31140627057263348551498094, 78905162375544758325347534, 68648809775830894573699528, 27796094893603291570999031, 78112521103814979835264292, 71297227107539993495048581, 5454556439229721216822295, 34681014828399246267433987, 1131661496461019698848506, 19470946870768387275356752, 28023327100560354609957208, 100760725121150163662605029, 47438315827128855974764291, 22463776763940393335995788, 57476506252950753777492233, 38279636483214683683465573, 72876847482914625705898507, 92908295887796648693653575, 11507826156847169190868322, 100199642765699100371989608, 26514294093065170664931843, 71477976132429316196420873, 3805079142319255203702393]


All that's left to do is find a vector equivalent to our target (0, ..., 0, 1/2).

In [6]:
M = matrix(32)
M[0, 0] = order
M[1:-1, 1:-1] = identity_matrix(30)
M[1:-1, 0] = -vector(dlogs)
M[-1, -2] = mod(1/2, order)
M[-1, -1] = 2^1024 # Kannan embedding with large weight

# .change_ring(ZZ).LLL()
print(M.LLL()[-1][:-1])

(2, -4, 1, 0, -1, -2, -2, -2, -2, 0, -1, -1, -4, 0, -1, 4, 1, -1, 2, -4, 0, 2, -7, -5, 1, 0, 2, 1, -1, 0, 0)


And that's pretty much it! We can pipe this private key into the challenge to get the flag.
```
> sage sqrt163.sage
Enter private key: (2, -4, 1, 0, -1, -2, -2, -2, -2, 0, -1, -1, -4, 0, -1, 4, 1, -1, 2, -4, 0, 2, -7, -5, 1, 0, 2, 1, -1, 0, 0)
Here is your flag: SEKAI{isogenni_where?_its_all_ideal_classes!}
```